# Plan-and-Execute

This notebook shows how to create a "plan-and-execute" style agent. This is heavily inspired by the [Plan-and-Solve](https://arxiv.org/abs/2305.04091) paper as well as the [Baby-AGI](https://github.com/yoheinakajima/babyagi) project.

The core idea is to first come up with a multi-step plan, and then go through that plan one item at a time.
After accomplishing a particular task, you can then revisit the plan and modify as appropriate.


The general computational graph looks like the following:


![plan-and-execute diagram](./img/plan-and-execute.png)


This compares to a typical [ReAct](https://arxiv.org/abs/2210.03629) style agent where you think one step at a time.
The advantages of this "plan-and-execute" style agent are:

1. Explicit long term planning (which even really strong LLMs can struggle with)
2. Ability to use smaller/weaker models for the execution step, only using larger/better models for the planning step


The following walkthrough demonstrates how to do so in LangGraph. The resulting agent will leave a trace like the following example: ([link](https://smith.langchain.com/public/d46e24d3-dda6-44d5-9550-b618fca4e0d4/r)).

## Setup

First, we need to install the packages required.

In [8]:
!pip install --quiet -U langchain langchain_openai
!pip install langchain_experimental
!pip install openai
!pip install numexpr
!pip install langchainhub
!pip install langgraph


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


Next, we need to set API keys for OpenAI (the LLM we will use) and Tavily (the search tool we will use)

Optionally, we can set API key for LangSmith tracing, which will give us best-in-class observability.

In [2]:
import os
from langchain.chat_models import ChatOpenAI
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.llms import OpenAI
from langchain.agents.tools import Tool
from langchain import LLMMathChain

In [3]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Define Tools

We will first define the tools we want to use. For this simple example, we will use a built-in search tool via Tavily. However, it is really easy to create your own tools - see documentation [here](https://python.langchain.com/docs/modules/agents/tools/custom_tools) on how to do that.

In [4]:
from lg_utils.cities.chains import HttpRequestChain
from lg_utils.rag.chains import HttpRequestChainRag
llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
requests_chain = HttpRequestChain()
rag_chain = HttpRequestChainRag()
tools = [
    Tool(
        name = "Search",
        func=requests_chain._call,
        description="Get the cities based on locations (for example countries or continents), population, etc.."
    ),
    Tool(
        name="Rag",
        func=rag_chain._call,
        description="Default plugin to call when no other plugin can be used. Any information not provided by other functions are meant to be retrieved from here."
    ),
]

model = ChatOpenAI(temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

/home/mvillanueva/.pyenv/versions/3.11.5/envs/sk_demo_env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/mvillanueva/.pyenv/versions/3.11.5/envs/sk_demo_env/lib/python3.11/site-packages/certifi/cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/mvillanueva/.pyenv/versions/3.11.5/envs/sk_demo_env/lib/python3.11/site-packages/certifi/cacert.pem'
INFO:numexpr.utils:Note: NumExpr detected 16 

## Define our Execution Agent

Now we will create the execution agent we want to use to execute tasks. 
Note that for this example, we will be using the same execution agent for each task, but this doesn't HAVE to be the case.

In [6]:
from langchain import hub
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner

from langchain.agents import create_openai_functions_agent
from langchain_openai import ChatOpenAI

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
planner = load_chat_planner(model)
# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-4-turbo-preview")
# executor = load_agent_executor(model, tools, verbose=True)
# agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)
# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.hub.langchain.com:443
DEBUG:urllib3.connectionpool:https://api.hub.langchain.com:443 "GET /commits/hwchase17/openai-functions-agent/?limit=100&offset=0 HTTP/1.1" 200 1457
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.hub.langchain.com:443
DEBUG:urllib3.connectionpool:https://api.hub.langchain.com:443 "GET /commits/hwchase17/openai-functions-agent/a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5 HTTP/1.1" 200 1215
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/mvillanueva/.pyenv/versions/3.11.5/envs/sk_demo_env/lib/python3.11/site-packages/certifi/cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/mvillanueva/.pyenv/versions/3.11.5/envs/sk_demo_env/lib/python3.11/site-packages/certifi/cacert.pem'


In [9]:
from langgraph.prebuilt import create_agent_executor

In [10]:
agent_executor = create_agent_executor(agent_runnable, tools)

In [11]:
agent_executor.invoke(
    {"input": "who is the winnner of the us open", "chat_history": []}
)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': 'You are a helpful assistant', 'role': 'system'}, {'content': 'who is the winnner of the us open', 'role': 'user'}], 'model': 'gpt-4-turbo-preview', 'functions': [{'name': 'Search', 'description': 'Get the cities based on locations (for example countries or continents), population, etc..', 'parameters': {'properties': {'__arg1': {'title': '__arg1', 'type': 'string'}}, 'required': ['__arg1'], 'type': 'object'}}, {'name': 'Rag', 'description': 'Default plugin to call when no other plugin can be used. Any information not provided by other functions are meant to be retrieved from here.', 'parameters': {'properties': {'__arg1': {'title': '__arg1', 'type': 'string'}}, 'required': ['__arg1'], 'type': 'object'}}], 'n': 1, 'stream': False, 'temperature': 0.7}}
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=Non

{'input': 'who is the winnner of the us open',
 'chat_history': [],
 'agent_outcome': AgentFinish(return_values={'output': "I encountered an error while trying to fetch the latest information. Could you specify which category you're interested in? For example, are you asking about the men's singles, women's singles, doubles, or another category in the US Open?"}, log="I encountered an error while trying to fetch the latest information. Could you specify which category you're interested in? For example, are you asking about the men's singles, women's singles, doubles, or another category in the US Open?"),
 'intermediate_steps': [(AgentActionMessageLog(tool='Rag', tool_input='us open winner 2023', log='\nInvoking: `Rag` with `us open winner 2023`\n\n\n', message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"us open winner 2023"}', 'name': 'Rag'}}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 127, 'total_tokens': 

## Define the State

Let's now start by defining the state the track for this agent.

First, we will need to track the current plan. Let's represent that as a list of strings.

Next, we should track previously executed steps. Let's represent that as a list of tuples (these tuples will contain the step and then the result)

Finally, we need to have some state to represent the final response as well as the original input.

In [12]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Tuple, Annotated, TypedDict
import operator


class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: str

## Planning Step

Let's now think about creating the planning step. This will use function calling to create a plan.

In [13]:
from langchain_core.pydantic_v1 import BaseModel


class Plan(BaseModel):
    """Plan to follow in future"""

    steps: List[str] = Field(
        description="different steps to follow, should be in sorted order"
    )

In [14]:
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.prompts import ChatPromptTemplate

planner_prompt = ChatPromptTemplate.from_template(
    """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

{objective}"""
)
planner = create_structured_output_runnable(
    Plan, ChatOpenAI(model="gpt-4-turbo-preview", temperature=0), planner_prompt
)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/mvillanueva/.pyenv/versions/3.11.5/envs/sk_demo_env/lib/python3.11/site-packages/certifi/cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/mvillanueva/.pyenv/versions/3.11.5/envs/sk_demo_env/lib/python3.11/site-packages/certifi/cacert.pem'


In [15]:
planner.invoke(
    {"objective": "what is the hometown of the current Australia open winner?"}
)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': 'For the given objective, come up with a simple step by step plan. This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.\n\nwhat is the hometown of the current Australia open winner?', 'role': 'user'}], 'model': 'gpt-4-turbo-preview', 'function_call': {'name': '_OutputFormatter'}, 'functions': [{'name': '_OutputFormatter', 'description': 'Output formatter. Should always be used to format your response to the user.', 'parameters': {'type': 'object', 'properties': {'output': {'description': 'Plan to follow in future', 'type': 'object', 'properties': {'steps': {'description': 'different steps to follow, should be in sorted order', 'type

Plan(steps=['Identify the current year to determine the most recent Australia Open tournament.', "Search for the winner of the most recent Australia Open men's or women's singles tournament.", 'Find the birthplace or commonly known hometown of the identified winner.'])

## Re-Plan Step

Now, let's create a step that re-does the plan based on the result of the previous step.

In [16]:
from langchain.chains.openai_functions import create_openai_fn_runnable


class Response(BaseModel):
    """Response to user."""

    response: str


replanner_prompt = ChatPromptTemplate.from_template(
    """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the follow steps:
{past_steps}

Update your plan accordingly. If no more steps are needed and you can return to the user, then respond with that. Otherwise, fill out the plan. Only add steps to the plan that still NEED to be done. Do not return previously done steps as part of the plan."""
)


replanner = create_openai_fn_runnable(
    [Plan, Response],
    ChatOpenAI(model="gpt-4-turbo-preview", temperature=0),
    replanner_prompt,
)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/mvillanueva/.pyenv/versions/3.11.5/envs/sk_demo_env/lib/python3.11/site-packages/certifi/cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/mvillanueva/.pyenv/versions/3.11.5/envs/sk_demo_env/lib/python3.11/site-packages/certifi/cacert.pem'


## Create the Graph

We can now create the graph!

In [24]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
async def execute_step(state: PlanExecute):
    task = state["plan"][0]
    agent_response = await agent_executor.ainvoke({"input": task, "chat_history": []})
    return {
        "past_steps": (task, agent_response["agent_outcome"].return_values["output"])
    }


async def plan_step(state: PlanExecute):
    plan = await planner.ainvoke({"objective": state["input"]})
    return {"plan": plan.steps}


async def replan_step(state: PlanExecute):
    output = await replanner.ainvoke(state)
    if isinstance(output, Response):
        return {"response": output.response}
    else:
        return {"plan": output.steps}


def should_end(state: PlanExecute):
    if state["response"]:
        return True
    else:
        return False

In [25]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(PlanExecute)

# Add the plan node
workflow.add_node("planner", plan_step)

# Add the execution step
workflow.add_node("agent", execute_step)

# Add a replan node
workflow.add_node("replan", replan_step)

workflow.set_entry_point("planner")

# From plan we go to agent
workflow.add_edge("planner", "agent")

# From agent, we replan
workflow.add_edge("agent", "replan")

workflow.add_conditional_edges(
    "replan",
    # Next, we pass in the function that will determine which node is called next.
    should_end,
    {
        # If `tools`, then we call the tool node.
        True: END,
        False: "agent",
    },
)

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [26]:
from langchain_core.messages import HumanMessage

config = {"recursion_limit": 3}
inputs = {"input": "what is the hometown of the 2024 Australia open winner?"}
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(v)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'plan': ['Wait until the 2024 Australian Open concludes.', 'Identify the winner of the 2024 Australian Open.', "Research the winner's biography to find their hometown."]}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'past_steps': ('Wait until the 2024 Australian Open concludes.', "I'm unable to wait for real-time events or provide updates as they happen. However, I can offer information on how to stay updated with the 2024 Australian Open. Would you like some tips on where to find live scores, match schedules, and results?")}


GraphRecursionError: Recursion limit of 3 reachedwithout hitting a stop condition. You can increase the limitby setting the `recursion_limit` config key.

## Conclusion

Congrats on making a plan-and-execute agent! One known limitations of the above design is that each task is still executed in sequence, meaning embarassingly parallel operations all add to the total execution time. You could improve on this by having each task represented as a DAG (similar to LLMCompiler), rather than a regular list.